<a href="https://colab.research.google.com/github/QaziSaim/CrewPractice/blob/main/Concurrent_Query_Resolution_System_Using_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install crewai
!pip install crewai-tools
!pip install langchain_google_genai


In [40]:
import os
import asyncio

In [89]:
from google.colab import userdata
gemini_api=userdata.get('GEMINI_API')
serper_api = userdata.get('SERPER_API_KEY')


In [42]:
from crewai import Agent, Crew, Task, LLM, Process

In [79]:
llm1 = LLM(
    model='gemini/gemini-2.5-flash',
    temperature=0.7,
    api_key=gemini_api
)
llm2 = LLM(
    model='gemini/gemini-2.0-flash',
    temperature=0.5,
    api_key = gemini_api
)
llm3 = LLM(
    model='gemini/gemini-2.0-flash',
    temperature=0.3
)

In [80]:
query_resolution = Agent(
    llm=llm1,
    role = 'Query Resolver',
    backstory="An AI agent that resolves customer queries efficiently and professionally.",
    goal="Resolve customer queries accurately and provide helpful solutions.",
    verbose=True

)

In [81]:
summary_agent = Agent(
    llm=llm2,
    role="Summary Generator",
    backstory="An AI agent that summarizes the resolution of customer queries.",
    goal="Provide a concise summary of the query resolution process.",
    verbose=True
)


In [82]:
resolution_task = Task(
    description="Resolve the customer query: {query}.",
    expected_output="A detailed resolution for the customer query.",
    agent=query_resolution
)

summary_task = Task(
    description="Summarize the resolution of the customer query: {query}.",
    expected_output="A concise summary of the query resolution.",
    agent=summary_agent
)


In [83]:
async def execute_query(query: str):
    crew = Crew(
        agents=[query_resolution, summary_agent],
        tasks=[resolution_task, summary_task],
        process=Process.sequential,
        verbose=True
    )
    result = await crew.kickoff_async(inputs={"query": query})
    return result


In [84]:
async def handle_two_queries(query_1: str, query_2: str):
    # Run both queries concurrently
    results = await asyncio.gather(
        execute_query(query_1),
        execute_query(query_2)
    )
    return results


In [85]:
query_1 = "I am unable to log in to my account. It says 'Invalid credentials', but I am sure I am using the correct username and password."
query_2 = "The payment gateway is not working. Also, a weird error message is displayed. My card has been charged, but the transaction is not going through."


In [86]:
try:
    loop = asyncio.get_event_loop()
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)


In [87]:
# Check if the event loop is already running
if loop.is_running():
    # If the loop is running, use `nest_asyncio` to allow re-entrant event loops
    import nest_asyncio
    nest_asyncio.apply()


In [88]:
# Run the async function
results = loop.run_until_complete(handle_two_queries(query_1, query_2))

# Print the results
for i, result in enumerate(results):
    print(f"Result for Query {i+1}:")
    print(result)
    print("\n---\n")

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 67f5620f-93f9-4e35-9592-7089e053c5ea                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 339b7512-f426-447f-9472-a281ff7b2b65
    Status: Executing Task...╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution